---
layout: post
title:  "ResNet in Breast Cancer Diagnose"
date:   2023-04-17 10:14:54 +0700
categories: DeepLearning
---

# Introduction

Let's explore a paper on the application of deep neural network in breast cancer screening: Deep Neural Networks Improve Radiologists' Performance in Breast Cancer Screening. A deep neural net with two stages is trained to learn the global and local information. This strategy allows the model to achieves a human competitive performance and the heatmaps indicating locations of findings. The authors propose a novel variant of a ResNet designed for medical imaging, with balance of depth and width. This allows the processing of large image and maintaining a reasonable amount of memory consumption.

# Dataset
The dataset includes 230000 digital screening mamography exams, with 1 million images from 141000 patients. Each exam has at least four images, corresponding to the four standard views in screening mammography: R-CC (right craniocaudal), L-CC (left craniocaudal), R-MLO (right mediolateral oblique), L-MLO (left mediolateral oblique). Here are some examples of those images: 

<img src="https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/42/9055242/8861376/wu1-2945514-small.gif">

The author also relies on diagnose of malignant or benign from bipopsies. There are 5800 exams with at least one biopsy performed within 120 days of the screening mammogram. The biopsies confirm malignant for 985 (8.4%) breasts and benign for 5500 (47.6%) breasts. 234 (2%) breasts have both malignant and benign findings. Example of labelling by a radiologist, with red being malignant and green as benign:

<img src="https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/42/9055242/8861376/wu2ab-2945514-small.gif">

# Multi task learning

Multi task learning is training a single neural network to do multiple tasks at the same time: to identify / classify multiple labels in one image. For example, for one single image, the algorithm would answer questions such as "is it a dress? is it jean? does it have green color? does it have red color?". The output for each task could be 0 or 1 (0 for no and 1 for yes). The total output would be a vector of 4 dimensions, 2 elements for product and 2 elements for colors. For example, a red dress would be [1 0 0 1]. In the paper, for each breast there are two binary labels: the absence/presence of malignant findings  and the absence/presence of benign findings. Since we have left and right breasts, each exam has a total of four binary labels. The predictions are denoted: $$ \hat{y}_{R,m}, \hat{y}_{L,m}, \hat{y}_{R,b}, \hat{y}_{L,b} $$. In which the prediction of benign findings serves as an auxiliary task to regularize the main task of predicting a malignant finding. Four input images are denoted $$ x_{R-CC}, x_{L-CC}, x_{R-MLO}, x_{L-MLO} $$. Each CC image is cropped at 2677x1942 and each MLO is cropped at 2974 x 1748.

# ResNet

They have for model architechture: the view wise model that concatenates L-CC and R-CC, and L-MLO and R-MLO. Then it makes prediction for CC and MLO, then average those. The image wise model makes four predictions independently and then averaged. The side wise model concatenate L-CC and L-MLO, then R-CC and R-MLO then predict for left and right breasts. The joint model concatenates all four views and jointly predict malignant and benign findings for both breasts.

<img src="https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/42/9055242/8861376/wu5-2945514-small.gif">

All models use ResNet-22 layers to output a fixed dimension hidden represnetation for each view and then two dense layers to map the hidden representations to the output predictions. What ResNet does is that it adds an intermediate input to the output of a series of convolution blocks. 

<img src="https://datagen.kinsta.cloud/app/uploads/2022/07/image1-3-1024x418.png">

This layer is simply F(x) + x itself and it is called a residual block since the mapping H(x) - (x) = F(x). This helps in the problem of vanishing gradient problem and adds original information skipping layers.

The author also tie weights for L-CC and R-CC, as well as L-MLO and R-MLO. Since they tie weights, they flip the left breast images before feeding into the model, so that all the breasts are rightward oriented. Weight tying is simply using the same weight matrix, and it is used as a regularization, to combat overfitting.

For the architecture, the first layer has 16 channels, with five ResNet blocks, each block doubles the number of channels, resulting in the last layer having 256 channels. The model is trained using Adam optimization algorithm, with a learning rate of $$ 10^{-5} $$ and a mini batch of size 4. $$ L_2 $$ regularization is applied to the weights with a coeffient of $$ 10^{-4.5} $$. The model has 6 million trainable parameters. They early stopped the training when the average of AUC did not improve for 20 epochs. 

# AUC

ROC curve stands for receiver operating characteristic curve is a graph showing the perfomance of a classification model at different classification threshold, by plotting two parameters: true positive rate and false positive rate. True positive rate (TPR) is recall and it is equal to $$ TPR = \frac{TP}{TP+FN} $$, false positive rate is defined as $$ FPR = \frac{FP}{FP + TN} $$. 

<img src="https://developers.google.com/static/machine-learning/crash-course/images/ROCCurve.svg">

AUC is the area under the ROC curve.

<img src="https://developers.google.com/static/machine-learning/crash-course/images/AUC.svg">

AUC is preferred in some cases since it is scale invariant. It measures how well the predictions are. Plus it is also classification threshold invariant, it measures the quality of the model's predictions irrespective of the threshold.

# Loss function

The prediction for each target is:

$$ \hat{y}_{R,m}(x_{R-CC}, x_{L-CC}, x_{R-MLO}, x_{L-MLO} = \frac{1}{2} \hat{y}_{R,m}^{CC} (x_{R-CC}, x_{L0CC}) + \frac{1}{2} \hat{y}_{R,m}^{MLO} (x_{R-MLO}, x_{L-MLO}), $$

and the loss function using binary cross entropy is:

$$ L(y_{R,m}, y_{L,m}, y_{R,b}, y_{L,b}, x_{R-CC}, x_{L-CC}, x_{R-MLO}, x_{L-MLO} = l(y_{R,m}, \hat{y}_{R,m}^{CC} (x_{R-CC}, x_{L-CC})) + ... $$


They use predictions from radiologists to compare results. 14 readers each reading 740 exams (1480 breasts): 368 exams randomly selected fromt he biopsied and 372 randomly selected. Exams are shuffled and readers give points on a scale of 0% to 100% for each breasts. The model achieves an AUC of 0.876 and the readers achieve from 0.705 to 0.860. So the authors evaluate a hybrid model of human-machine, with the formular $$ \hat{y}_{hybrid} = \lambda \hat{y}_{radiologist} + (1 - \lambda \hat{y}_{model} $$. With $$ \lambda = 0.5 $$, the hybrid achieves an AUC of 0.891, higher than as they do individually. This suggest the use of machine learning as a tool to assist radiologists. 

<img src="https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/42/9055242/8861376/wu9ab-2945514-small.gif">

Example of one prediction, the following image has {'benign': 0.06533989310264587, 'malignant': 0.005473622120916843}

![result](https://user-images.githubusercontent.com/7457301/233016859-23093f9d-dd9d-470d-8bee-ff9cacfba64a.png)

